In [ ]:
import sys
import os
import numpy as np


In [ ]:
from DiffusionAnalysis.loaders import DatDirectoryStructureLoader
from DiffusionAnalysis.analysis import TracerMSDAnalyser
from DiffusionAnalysis.trajectory import DisplacementTrajectory




In [ ]:
# your_code.py
from DiffusionAnalysis.examples.config import FILE_PATH_SECRET

loader = DatDirectoryStructureLoader(FILE_PATH_SECRET, structures_slice=slice(2500, 4000, 1), md_timestep=0.1, md_time_unit='ps')

In [ ]:
traj = DisplacementTrajectory(loader)

In [ ]:
# traj.generate_displacement_trajectory(show_progress=True)
# # #save as a pickle (not implemented in the class)
import pickle
# with open('displacement_traj.pickle', 'wb') as f:
#      pickle.dump(traj, f)

#load the pickle
with open('displacement_traj.pickle', 'rb') as f:
    traj = pickle.load(f)
    


In [ ]:
msd_analysis = TracerMSDAnalyser(traj)

# Assuming you have an instance of MSDAnalysis called 'msd_analysis'

# Calculate the 3D MSD
msd_3d = msd_analysis.calculate_msd(tracer_specs=['H'], return_3d_msd=True)

msd_3d_framework = msd_analysis.calculate_msd(tracer_specs=['H'], framework_specs=['He'], correct_drift=True, return_3d_msd=True)
fig1 = msd_analysis.plot_msd((msd_3d), title='3D MSD Plot (Drift Corrected)', legend_loc='upper left', skip_points=4)
fig1_log = msd_analysis.plot_msd((msd_3d), title='3D MSD Plot (Log Scale) (Drift Corrected)', legend_loc='upper left', skip_points=4, log_scale=True)


#plot 3d vs 3d framework drift corrected msd
difference = msd_3d - msd_3d_framework
fig1_diff = msd_analysis.plot_msd((difference), title='3D MSD vs 3D MSD (Drift Corrected) Error', legend_loc='upper left', skip_points=4)


# Calculate the MSD for each direction (x, y, z)
msd_x, msd_y, msd_z = msd_analysis.calculate_msd(tracer_specs=['H'],return_3d_msd=False)

# Plot the MSD for each direction
fig2 = msd_analysis.plot_msd((msd_x, msd_y, msd_z), labels=['x','y' ,'z'], title='X,Y,Z MSD Plot', legend_loc='upper left', skip_points=4)
fig2.show()
# Calculate and plot the MSD along a non-normalized lattice vector for host atoms, correcting for framework drift
lattice_vector = np.array([2, 0, 0])  # Example non-normalized lattice vector
msd_along_lattice = msd_analysis.calculate_msd(lattice_vector = lattice_vector)
fig3 = msd_analysis.plot_msd(msd_along_lattice, labels=f'MSD Along Lattice Vector {lattice_vector}', title='MSD Along Lattice Vector [2,0,0] (Drift Uncorrected)', skip_points=4)
fig3.show()


In [ ]:
from DiffusionAnalysis.analysis import VanHoveAnalyser
import numpy as np
import matplotlib.pyplot as plt


# Create a VanHoveAnalysis object
van_hove_analyzer = VanHoveAnalyser(traj)

# Set the parameters for the analysis
tau_values = [1, 5, 10, 500]  # Time lags in number of timesteps
r_range = (0, 5)  # Range of distances to consider
n_bins = 100  # Number of bins for the histogram

self_data = []
self_data = van_hove_analyzer.calculate_van_hove(tau_values, r_range, n_bins, mode='self',type_a_specs=['H'])

self_fig = van_hove_analyzer.plot_van_hove(self_data, mode='self', title='Self-part of the Van Hove Correlation Function')

import tracemalloc


distinct_data = []
distinct_data = van_hove_analyzer.calculate_van_hove(tau_values, r_range, n_bins, mode='distinct', type_a_specs=['H'], type_b_specs=['He'])

# Plot the distinct-part of the van Hove correlation function
distinct_fig = van_hove_analyzer.plot_van_hove(distinct_data, mode='distinct', title='Distinct-part of the Van Hove Correlation Function', include_normalization=True)


In [ ]:
from DiffusionAnalysis.analysis import TMSDAnalyser

# Create an instance of the tMSDAnalysis class
tmsd_analysis = TMSDAnalyser(traj)

# Define the minimum and maximum time lag values (in steps) and the number of points
min_tau = 1
max_tau = 1000
num_points = 100

# Calculate tMSD
time_lag_values, tMSD_values = tmsd_analysis.calculate_tMSD(min_tau, max_tau, num_points)

# Plot the tMSD data
temperature = 500  # Example temperature value
label = f'{temperature} K'
fig = tmsd_analysis.plot_tMSD(time_lag_values, tMSD_values, label)

fig2 = tmsd_analysis.plot_tMSD_exponent(time_lag_values, tMSD_values, label, average_window_size=3) 




In [ ]:
time_lags = [1, 100,500]  # Example time lags
num_bins = 150

fig = msd_analysis.plot_displacement_distribution(tracer_specs=['H'],
                                                  time_lags=time_lags,
                                                  num_bins=num_bins, use_3d=False, lattice_vector=[1, 0, 0])
fig.gca().set_xlim(-3, 3)

fig = msd_analysis.plot_displacement_distribution(tracer_specs=['H'],
                                                  time_lags=time_lags,
                                                  num_bins=num_bins,
                                                  use_3d=True,)

fig.gca().set_xlim(0, 6)

plt.show()